In [1]:
!nvidia-smi

Tue Mar 21 19:39:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.94       Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| 45%   34C    P0    N/A /  75W |    591MiB /  4096MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load Dataset

In [2]:
import pandas as pd

df = pd.read_csv("../dataset/gmaps_review_dataset.csv")

In [3]:
df.head()

,_id,datetime,location,rating,scrapped_at,text
0,47d0cdd937754bd6b860f89b2bab1dbb,2022-05-15 11:58:43,Curug Malela,4,2023-02-15 11:58:43,Akses jalannya waktu itu masih sulit di jangka...
1,4804acd6c05e4f89b098e2ca35019419,2022-08-15 11:58:43,Curug Malela,5,2023-02-15 11:58:43,"Perjalanan yg bnr"" bikin Syahduu ,, dr Tempat ..."
2,3eae265bf32a45eca31765a4145bc030,2022-03-15 11:58:43,Curug Malela,5,2023-02-15 11:58:43,"Minggu 13 februari 2022 ,\n\ngas santai pakai ..."
3,61037dbdb7b14045be49d4494e95cf05,2022-05-15 11:58:44,Curug Malela,5,2023-02-15 11:58:44,7 mei 2022\nTouring bari mudik\nMntap perjalan...
4,a2c9e817e2b949c6880f971f43a11d2f,2022-08-15 11:58:44,Curug Malela,5,2023-02-15 11:58:44,Perjalanan touring motor dari bekasi melewati ...


In [4]:
df['datetime'] = df['datetime'].apply(pd.to_datetime)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13587 entries, 0 to 13586
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   _id          13587 non-null  object        
 1   datetime     13587 non-null  datetime64[ns]
 2   location     13587 non-null  object        
 3   rating       13587 non-null  int64         
 4   scrapped_at  13587 non-null  object        
 5   text         13587 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 637.0+ KB


# Masking Dataset

In [6]:
mask = (df['datetime'] >= '2019-1-1')
df = df.loc[mask]

In [7]:
len(df)

12569

In [8]:
df = df.drop(columns=['rating', 'scrapped_at'])

## Drop Duplicate

In [9]:
df = df.drop_duplicates(subset=['text'])

In [10]:
len(df)

11937

In [11]:
# duplicate = df[df.duplicated(subset=['text'])]

In [12]:
# duplicate

# Analysis

# Data Preprocessing

## Cleaning

In [13]:
def get_translate_text(text):
    keyword = ['(Diterjemahkan oleh Google)', '(Asli)']
    if keyword[0] in text and keyword[1] in text:
        text = text[text.find(keyword[0]) + len(keyword[0]):text.rfind(keyword[1])]
    elif keyword[0] in text and keyword[1] not in text:
        text = text[text.find(keyword[0]) + len(keyword[0]):]
    return text 

In [14]:
import re

def clean_text(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r' ', text) # no emoji
    text = text.lower() #to lowercase
    text = re.sub('\n', ' ', text) #remove \n
    text = re.sub(r'[^\w\s]', ' ', text) #remove punctuation
    text = re.sub("\d+", "", text) #remove number
    text = re.sub(' +', ' ', text) #remove multiple whitespace
    
    if re.search(r'([a-zA-Z])\1{2,}', text): # Spell
        text = re.sub(r'([a-zA-Z])\1{2,}','\\1', text)
        
    return text

In [15]:
df['text_preprocess'] = df['text'].apply(get_translate_text)
df['text_preprocess'] = df['text_preprocess'].apply(clean_text)
df['len_char'] = df['text_preprocess'].apply(lambda x: len(x))
df['len_word'] = df['text_preprocess'].apply(lambda x: len(x.split(' ')))

In [16]:
df.head()

,_id,datetime,location,text,text_preprocess,len_char,len_word
0,47d0cdd937754bd6b860f89b2bab1dbb,2022-05-15 11:58:43,Curug Malela,Akses jalannya waktu itu masih sulit di jangka...,akses jalannya waktu itu masih sulit di jangka...,84,13
1,4804acd6c05e4f89b098e2ca35019419,2022-08-15 11:58:43,Curug Malela,"Perjalanan yg bnr"" bikin Syahduu ,, dr Tempat ...",perjalanan yg bnr bikin syahduu dr tempat park...,110,19
2,3eae265bf32a45eca31765a4145bc030,2022-03-15 11:58:43,Curug Malela,"Minggu 13 februari 2022 ,\n\ngas santai pakai ...",minggu februari gas santai pakai motor dari da...,1993,338
3,61037dbdb7b14045be49d4494e95cf05,2022-05-15 11:58:44,Curug Malela,7 mei 2022\nTouring bari mudik\nMntap perjalan...,mei touring bari mudik mntap perjalanan sungg...,162,25
4,a2c9e817e2b949c6880f971f43a11d2f,2022-08-15 11:58:44,Curug Malela,Perjalanan touring motor dari bekasi melewati ...,perjalanan touring motor dari bekasi melewati ...,152,23


In [17]:
df[df['_id'] == '3452054e89614078a257967adbbee4b6']

,_id,datetime,location,text,text_preprocess,len_char,len_word
8340,3452054e89614078a257967adbbee4b6,2021-02-15 14:13:20,Sanghyang Kenit,Kerennnn 😍 …,keren,6,2


## Tokenizing

In [19]:
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

In [20]:
import nltk
from nltk.tokenize import word_tokenize

df['tokens'] = df['text_preprocess'].apply(lambda x: word_tokenize(x))

In [21]:
df['tokens']

0        [akses, jalannya, waktu, itu, masih, sulit, di...
1        [perjalanan, yg, bnr, bikin, syahduu, dr, temp...
2        [minggu, februari, gas, santai, pakai, motor, ...
3        [mei, touring, bari, mudik, mntap, perjalanan,...
4        [perjalanan, touring, motor, dari, bekasi, mel...
                               ...                        
13582    [hanya, mencoba, sarapan, dan, offroad, pakai,...
13583    [pngnnya, ksni, petik, strawberry, tpi, lgi, g...
13584    [ksini, cuma, main, aja, dan, lihat, lihat, su...
13585    [luar, biasa, mantap, makan, prasmanan, ny, mu...
13586             [tempat, dan, suasananya, sejuk, banget]
Name: tokens, Length: 11937, dtype: object

## Stopwords removal

In [22]:
from nltk.corpus import stopwords 

stop_words = stopwords.words('indonesian')
stop_words.extend(['yg', 'rb', 'nya'])

df['tokens'] = df['tokens'].apply(lambda x: [w for w in x if not w in stop_words])

In [23]:
df['tokens']

0        [akses, jalannya, sulit, jangkau, perjalanan, ...
1        [perjalanan, bnr, bikin, syahduu, dr, parkir, ...
2        [minggu, februari, gas, santai, pakai, motor, ...
3        [mei, touring, bari, mudik, mntap, perjalanan,...
4        [perjalanan, touring, motor, bekasi, melewati,...
                               ...                        
13582    [mencoba, sarapan, offroad, pakai, land, rover...
13583    [pngnnya, ksni, petik, strawberry, tpi, lgi, g...
13584    [ksini, main, aja, lihat, lihat, suasananya, e...
13585                [mantap, makan, prasmanan, ny, murah]
13586                          [suasananya, sejuk, banget]
Name: tokens, Length: 11937, dtype: object

## Stemming

In [24]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()
 
df['tokens'] = df['tokens'].apply(lambda x: stemmer.stem(' '.join(x)).split(' '))

In [25]:
df['tokens']

0        [akses, jalan, sulit, jangkau, jalan, tuji, cu...
1        [jalan, bnr, bikin, syahduu, dr, parkir, lokas...
2        [minggu, februari, gas, santai, pakai, motor, ...
3        [mei, touring, bari, mudik, mntap, jalan, sung...
4        [jalan, touring, motor, bekas, lewat, track, s...
                               ...                        
13582    [coba, sarap, offroad, pakai, land, rover, mak...
13583    [pngnnya, ksni, petik, strawberry, tpi, lgi, g...
13584    [ksini, main, aja, lihat, lihat, suasana, emg,...
13585                [mantap, makan, prasmanan, ny, murah]
13586                             [suasana, sejuk, banget]
Name: tokens, Length: 11937, dtype: object

In [26]:
df[df['_id'] == '9d6e15357c5441078e33cf373d9b272d']["tokens"]
# df[df['_id'] == '9d6e15357c5441078e33cf373d9b272d']

1049    [bagus, hiking, teman, keluarga, warung, jual,...
Name: tokens, dtype: object

In [27]:
len_char = len(df.iloc[0]['text_preprocess'])
len_word = len(df.iloc[0]['text_preprocess'].split(' '))
len_char, len_word

(84, 13)

# Explanatory Data Analysis

In [28]:
len(df)

11937

In [29]:
df[df.duplicated(subset=['text_preprocess'])]

,_id,datetime,location,text,text_preprocess,len_char,len_word,tokens
876,394f0b5314144eceb2265fe283b817f6,2022-02-15 12:25:50,Curug Tilu Leuwi Opat,Good\n\n(Diterjemahkan oleh Google)\nBagus,bagus,6,2,[bagus]
877,f1a148b5936b49aa96ba495c993899d5,2021-02-15 12:25:50,Curug Tilu Leuwi Opat,Ok\n\n(Diterjemahkan oleh Google)\nOke,oke,4,2,[oke]
880,1018e45b7efd4ad0aa94701414f10b0e,2019-02-15 12:25:50,Curug Tilu Leuwi Opat,Nice place\n\n(Diterjemahkan oleh Google)\nTem...,tempat yang bagus,18,4,[bagus]
883,4b33a656a0c44977b6921657dc28366c,2019-02-15 12:25:51,Curug Tilu Leuwi Opat,(Diterjemahkan oleh Google) Bagus\n\n(Asli)\nB...,bagus,7,3,[bagus]
894,600dcccf0e1a45e38327de12c429f79a,2020-02-15 12:25:53,Curug Tilu Leuwi Opat,(Diterjemahkan oleh Google) Bagus\n\n(Asli)\nجيد,bagus,7,3,[bagus]
...,...,...,...,...,...,...,...,...
11716,d6378cac0ae1467396cfc8607552f83b,2019-02-23 18:13:41,Ciwangun Indah Camp (CIC),(Diterjemahkan oleh Google) Luar biasa\n\n(Asl...,luar biasa,12,4,[]
11726,b6f54fd2055c455e9f51372b78047599,2021-02-23 18:13:42,Ciwangun Indah Camp (CIC),(Diterjemahkan oleh Google) Bagus....\n\n(Asli...,bagus,7,3,[bagus]
11727,289892de5ca24de6a88bb6a43ea1b089,2020-02-23 18:13:42,Ciwangun Indah Camp (CIC),(Diterjemahkan oleh Google) Kembali ke alam\n\...,kembali ke alam,17,5,[alam]
12609,8598a4886ed54ec3926dbc49f80b3bfd,2020-02-23 18:35:13,Orchid Forest Cikole,Terima Kasih banyak Kak atas kunjungan dan rev...,terima kasih banyak kak atas kunjungan dan rev...,93,14,"[terima, kasih, kak, kunjung, reviewnya, tungg..."


In [30]:
df = df.drop_duplicates(subset=['text_preprocess'])

In [31]:
df['len_token'] = df['tokens'].apply(lambda x: len(x))

In [32]:
df[df['len_token'] <= 3]

,_id,datetime,location,text,text_preprocess,len_char,len_word,tokens,len_token
49,457feb4e6161474f829dd51b07af05fe,2023-01-15 11:58:46,Curug Malela,Niagara kecil yang tersembunyi,niagara kecil yang tersembunyi,30,4,"[niagara, sembunyi]",2
79,7f59834407cf4fddaf1df89ad19086df,2021-02-15 11:58:47,Curug Malela,Kami siap mengantar dengan menggunakan Landrover,kami siap mengantar dengan menggunakan landrover,48,6,"[antar, landrover]",2
80,99b21520cfe74ac5a7bf488dbd835994,2022-12-15 11:58:47,Curug Malela,Sangat bermakna alam kita,sangat bermakna alam kita,25,4,"[makna, alam]",2
86,9633aebda8c14ee5a9221c3f614ccc94,2021-02-15 11:58:48,Curug Malela,Rasa capenya bisa tergantikan🖤 …,rasa capenya bisa tergantikan,30,5,"[cape, ganti]",2
91,bbbe2c7a216a498ab399a5ce2aba4213,2022-09-15 11:58:48,Curug Malela,Mulai banyak renov saya suka,mulai banyak renov saya suka,28,5,"[renov, suka]",2
...,...,...,...,...,...,...,...,...,...
13525,0af9f3525abd4368862a3d208be8b7bc,2019-02-23 18:23:20,Terminal Wisata Grafika Cikole,Pemandangan nya bagus bangeeettt,pemandangan nya bagus banget,28,4,"[pandang, bagus, banget]",3
13554,57d9d3c198284feda8adf766745fc86c,2022-12-23 18:23:25,Terminal Wisata Grafika Cikole,Pilihan tepat untuk liburan 👍👍👍 …,pilihan tepat untuk liburan,28,5,"[pilih, libur]",2
13561,551121bcddf94895b3e91ad898fe5cb8,2020-02-23 18:23:26,Terminal Wisata Grafika Cikole,Mantaaapppp lah tempatnya,mantap lah tempatnya,20,3,"[mantap, tempat]",2
13571,c229fba45de2445d910bad68c03b8ada,2021-02-23 18:23:27,Terminal Wisata Grafika Cikole,Nyaman tuk bersantai,nyaman tuk bersantai,20,3,"[nyaman, tuk, santa]",3


In [33]:
df = df[df['len_token'] >= 2]

In [34]:
len(df)

11548

In [35]:
# Import the wordcloud library
from wordcloud import WordCloud
# Join the different processed titles together.
long_string = ','.join(list(df['tokens'].values))
# Create a WordCloud object
wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue')
# Generate a word cloud
wordcloud.generate(long_string)
# Visualize the word cloud
wordcloud.to_image()

TypeError: sequence item 0: expected str instance, list found

# Convert to corpus

In [36]:
import gensim.corpora as corpora

id2word = corpora.Dictionary(df['tokens'])
texts = df['tokens']
corpus = [id2word.doc2bow(text) for text in texts]

In [37]:
corpus[1]

[(1, 1),
 (2, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 2),
 (13, 1),
 (14, 1),
 (15, 1)]

In [38]:
from gensim import models
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [39]:
corpus_tfidf[1]

[(1, 0.1647762302799044),
 (2, 0.09674906051156616),
 (7, 0.21189178778325604),
 (8, 0.4406563752431694),
 (9, 0.22485323640086963),
 (10, 0.2653998486625921),
 (11, 0.4622986046447371),
 (12, 0.27727230806398445),
 (13, 0.11740930653459876),
 (14, 0.2230124917380367),
 (15, 0.49929624017673535)]

# Apply Model

In [47]:
import gensim
from pprint import pprint

num_topics = 7

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.025*"jalan" + 0.013*"bagus" + 0.012*"air" + 0.012*"curug" + 0.011*"masuk" '
  '+ 0.011*"parkir" + 0.009*"lokasi" + 0.009*"tiket" + 0.009*"luas" + '
  '0.008*"tempat"'),
 (1,
  '0.021*"bagus" + 0.017*"makan" + 0.014*"masuk" + 0.012*"tiket" + '
  '0.012*"jalan" + 0.010*"foto" + 0.009*"harga" + 0.009*"lumayan" + '
  '0.008*"air" + 0.008*"wahana"'),
 (2,
  '0.018*"bagus" + 0.016*"tempat" + 0.013*"jalan" + 0.011*"gunung" + '
  '0.010*"indah" + 0.010*"masuk" + 0.009*"banget" + 0.009*"pandang" + '
  '0.009*"kalo" + 0.009*"sejuk"'),
 (3,
  '0.018*"bagus" + 0.017*"foto" + 0.014*"anak" + 0.011*"wisata" + 0.011*"spot" '
  '+ 0.011*"tempat" + 0.009*"jalan" + 0.009*"alam" + 0.008*"masuk" + '
  '0.008*"main"'),
 (4,
  '0.019*"wisata" + 0.017*"foto" + 0.016*"masuk" + 0.015*"tiket" + '
  '0.012*"jalan" + 0.011*"parkir" + 0.010*"bagus" + 0.009*"pandang" + '
  '0.009*"keluarga" + 0.008*"cocok"'),
 (5,
  '0.020*"bagus" + 0.016*"foto" + 0.014*"cocok" + 0.013*"sejuk" + '
  '0.012*"indah" + 0.012*

# Evaluate

In [48]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.5020343978431774


In [44]:
for index, score in sorted(lda_model[corpus_tfidf[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.7311912178993225	 
Topic: 0.017*"masuk" + 0.016*"wisata" + 0.016*"jalan" + 0.015*"tiket" + 0.015*"foto" + 0.014*"bagus" + 0.009*"tempat" + 0.009*"makan" + 0.009*"parkir" + 0.008*"spot"

Score: 0.029872620478272438	 
Topic: 0.029*"jalan" + 0.020*"bagus" + 0.014*"parkir" + 0.012*"tempat" + 0.011*"lumayan" + 0.011*"banget" + 0.010*"masuk" + 0.009*"foto" + 0.008*"makan" + 0.008*"keluarga"

Score: 0.029870187863707542	 
Topic: 0.013*"makan" + 0.012*"banget" + 0.012*"masuk" + 0.010*"enak" + 0.009*"bagus" + 0.009*"anak" + 0.008*"tempat" + 0.008*"tiket" + 0.007*"foto" + 0.007*"alam"

Score: 0.029868215322494507	 
Topic: 0.023*"bagus" + 0.014*"pandang" + 0.013*"wisata" + 0.012*"masuk" + 0.012*"indah" + 0.011*"tempat" + 0.011*"jalan" + 0.011*"harga" + 0.010*"banget" + 0.010*"anak"

Score: 0.02986811473965645	 
Topic: 0.020*"bagus" + 0.014*"foto" + 0.011*"jalan" + 0.011*"wisata" + 0.011*"cocok" + 0.010*"tempat" + 0.009*"masuk" + 0.009*"indah" + 0.009*"lokasi" + 0.008*"spot"

Score: 0.02

In [ ]:
df.head()

In [43]:
df.to_excel("../dataset/dataset_preprocess.xlsx", index=False)

# Result

In [ ]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

# Sample Text

In [ ]:
text = "Bagus tempatnya pada saat"

In [ ]:
def spell(word):
    if re.search(r'([a-zA-Z])\1{2,}', word):
        word = re.sub(r'([a-zA-Z])\1{2,}','\\1', word)
    return word

In [ ]:
text_spell = spell(text)
text_spell

In [ ]:
df.iloc[6].text

In [ ]:
text = df.iloc[6].text

In [ ]:
text_preprocess = get_translate_text(text)
text_preprocess = clean_emoji(text_preprocess)
text_preprocess = clean_text(text_preprocess)

In [ ]:
text_preprocess

# Text Column

In [ ]:
df['text_preprocess'] = df['text'].apply(get_translate_text)
df['text_preprocess'] = df['text_preprocess'].apply(clean_emoji)
df['text_preprocess'] = df['text_preprocess'].apply(clean_text)

In [ ]:
df.iloc[15].text

In [ ]:
df.iloc[15].text_preprocess

In [ ]:
sample = df.loc[df['_id'] == '9d6e15357c5441078e33cf373d9b272d'].text_preprocess

In [ ]:
print(sample)